In [ ]:
!pip install -U spacy-nightly --pre
!pip install transformers
!python -m spacy download en_core_web_trf
!pip install -U spacy transformers
! pip install datasets 

In [ ]:
! cd rel_component

/bin/bash: line 0: cd: rel_component: No such file or directory


In [ ]:
import transformers
from datasets import load_dataset
import pandas as pd
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
import numpy as np


In [ ]:
dataset = load_dataset("ade_corpus_v2",'Ade_corpus_v2_drug_ade_relation')

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'effect', 'indexes'],
        num_rows: 6821
    })
})


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [ ]:
for index,row in enumerate(dataset["train"]):
  updated_text = row["text"].replace("-", " ")
  dataset["train"][index]["text"] = updated_text

In [ ]:
for row in dataset["train"]:
  if "-" in row["text"]:
    print(row["text"])

Intravenous azithromycin-induced ototoxicity.
Immobilization, while Paget's bone disease was present, and perhaps enhanced activation of dihydrotachysterol by rifampicin, could have led to increased calcium-release into the circulation.
Naproxen, the most common offender, has been associated with a dimorphic clinical pattern: a PCT-like presentation and one simulating erythropoietic protoporphyria in the pediatric population.
RESULTS: A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands.
RESULTS: A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands.
RESULTS: A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthr

In [ ]:
consolidated_dataset = {}

for row in dataset["train"]:
    if row["text"] in consolidated_dataset:
        start_drug = row["indexes"]["drug"]["start_char"]
        end_drug = row["indexes"]["drug"]["end_char"]
        start_effect = row["indexes"]["effect"]["start_char"]
        end_effect = row["indexes"]["effect"]["end_char"]

        for i,drug_start_index in enumerate(start_drug):
          if drug_start_index in consolidated_dataset[row["text"]]["drug_indices_start"]:
            index = consolidated_dataset[row["text"]]["drug_indices_start"].index(drug_start_index)
            lenght_existing = consolidated_dataset[row["text"]]["drug_indices_end"][index] - drug_start_index
            lenght_new = row["indexes"]["drug"]["end_char"][i] - drug_start_index
            if lenght_existing == lenght_new:
              continue
            elif lenght_new > lenght_existing :
              consolidated_dataset[row["text"]]["drug_indices_end"][index] = row["indexes"]["drug"]["end_char"][i]
              

          else:
            consolidated_dataset[row["text"]]["drug_indices_start"].append(drug_start_index)
            consolidated_dataset[row["text"]]["drug_indices_end"].append(row["indexes"]["drug"]["end_char"][i])
            consolidated_dataset[row["text"]]["effect_indices_start"].append(row["indexes"]["effect"]["start_char"])
            consolidated_dataset[row["text"]]["effect_indices_end"].append(row["indexes"]["effect"]["end_char"])
            consolidated_dataset[row["text"]]["drug"].append(row["drug"])
            consolidated_dataset[row["text"]]["effect"].append(row["effect"])

            consolidated_dataset[row["text"]]["drug_indices_start"] = flatten_list(consolidated_dataset[row["text"]]["drug_indices_start"])
            consolidated_dataset[row["text"]]["drug_indices_end"] = flatten_list(consolidated_dataset[row["text"]]["drug_indices_end"])
            consolidated_dataset[row["text"]]["effect_indices_start"] = flatten_list(consolidated_dataset[row["text"]]["effect_indices_start"])
            consolidated_dataset[row["text"]]["effect_indices_end"] = flatten_list(consolidated_dataset[row["text"]]["effect_indices_end"])
        
    else:
        consolidated_dataset[row["text"]] = {
            "text": row["text"],
            "drug": [row["drug"]],
            "effect": [row["effect"]],
            # use sets because the indices can repeat for various reasons
            "drug_indices_start": flatten_list(row["indexes"]["drug"]["start_char"]),
            "drug_indices_end": flatten_list(row["indexes"]["drug"]["end_char"]),
            "effect_indices_start": flatten_list(row["indexes"]["effect"]["start_char"]),
            "effect_indices_end": flatten_list(row["indexes"]["effect"]["end_char"])
        }



for row in dataset["train"]:
  consolidated_dataset[row["text"]]["drug_indices_start"] = sorted(set(consolidated_dataset[row["text"]]["drug_indices_start"]))
  consolidated_dataset[row["text"]]["drug_indices_end"] = sorted(set(consolidated_dataset[row["text"]]["drug_indices_end"]))
  consolidated_dataset[row["text"]]["effect_indices_start"] = sorted(set(consolidated_dataset[row["text"]]["effect_indices_start"]))
  consolidated_dataset[row["text"]]["effect_indices_end"] = sorted(set(consolidated_dataset[row["text"]]["effect_indices_end"]))


df = pd.DataFrame(list(consolidated_dataset.values()))


In [ ]:
consolidated_dataset["Here, we describe a case of Vogt-Koyanagi-Harada disease occurring 4 months after the start of interferon alpha treatment, probably induced by the immunomodulatory effects of interferon."]

{'text': 'Here, we describe a case of Vogt-Koyanagi-Harada disease occurring 4 months after the start of interferon alpha treatment, probably induced by the immunomodulatory effects of interferon.',
 'drug': ['interferon'],
 'effect': ['Vogt-Koyanagi-Harada disease'],
 'drug_indices_start': [95, 175],
 'drug_indices_end': [111, 185],
 'effect_indices_start': [28],
 'effect_indices_end': [56]}

In [ ]:
for row in dataset["train"]:
  string = "Here, we describe a case of Vogt-Koyanagi-Hara"
  if string in row["text"]:
    print(row)
    

{'text': 'Here, we describe a case of Vogt-Koyanagi-Harada disease occurring 4 months after the start of interferon alpha treatment, probably induced by the immunomodulatory effects of interferon.', 'drug': 'interferon', 'effect': 'Vogt-Koyanagi-Harada disease', 'indexes': {'drug': {'start_char': [95, 175], 'end_char': [105, 185]}, 'effect': {'start_char': [28], 'end_char': [56]}}}
{'text': 'Here, we describe a case of Vogt-Koyanagi-Harada disease occurring 4 months after the start of interferon alpha treatment, probably induced by the immunomodulatory effects of interferon.', 'drug': 'interferon alpha', 'effect': 'Vogt-Koyanagi-Harada disease', 'indexes': {'drug': {'start_char': [95], 'end_char': [111]}, 'effect': {'start_char': [28], 'end_char': [56]}}}


In [ ]:
df['len_effect'] = df['effect_indices_end'].str.len() - df['effect_indices_start'].str.len()
df['len_drug'] = df['drug_indices_end'].str.len() - df['drug_indices_start'].str.len()


In [ ]:
df[df['len_drug'] > 0]

,text,drug,effect,drug_indices_start,drug_indices_end,effect_indices_start,effect_indices_end,len_effect,len_drug


In [ ]:
df = df[df['len_drug'] == 0]
df = df[df['len_effect'] == 0]
df.drop(columns=['len_drug', 'len_effect'])
df

,text,drug,effect,drug_indices_start,drug_indices_end,effect_indices_start,effect_indices_end,len_effect,len_drug
0,Intravenous azithromycin-induced ototoxicity.,[azithromycin],[ototoxicity],[12],[24],[33],[44],0,0
1,"Immobilization, while Paget's bone disease was...",[dihydrotachysterol],[increased calcium-release],[91],[109],[143],[168],0,0
2,Unaccountable severe hypercalcemia in a patien...,[dihydrotachysterol],[hypercalcemia],[84],[102],[21],[34],0,0
3,METHODS: We report two cases of pseudoporphyri...,"[naproxen, oxaprozin]","[pseudoporphyria, pseudoporphyria]","[58, 71]","[66, 80]",[32],[47],0,0
4,"Naproxen, the most common offender, has been a...",[Naproxen],[erythropoietic protoporphyria],[0],[8],[134],[163],0,0
...,...,...,...,...,...,...,...,...,...
4266,Reversible nonthrombocytopenic palpable purpur...,[metoclopramide],[nonthrombocytopenic palpable purpura],[64],[78],[11],[47],0,0
4267,Lithium treatment was terminated in 1975 becau...,"[Lithium, lithium, Lithium, lithium]","[diabetes insipidus-like syndrome, diabetes in...","[0, 52]","[7, 59]","[52, 80]","[72, 112]",0,0
4268,Eosinophilia caused by clozapine was observed ...,[clozapine],[Eosinophilia],[23],[32],[0],[12],0,0
4269,Eosinophilia has been encountered from 0.2 to ...,[clozapine],[Eosinophilia],[55],[64],[0],[12],0,0


In [ ]:
df

,text,drug,effect,drug_indices_start,drug_indices_end,effect_indices_start,effect_indices_end,len_effect,len_drug
0,Intravenous azithromycin-induced ototoxicity.,[azithromycin],[ototoxicity],[12],[24],[33],[44],0,0
1,"Immobilization, while Paget's bone disease was...",[dihydrotachysterol],[increased calcium-release],[91],[109],[143],[168],0,0
2,Unaccountable severe hypercalcemia in a patien...,[dihydrotachysterol],[hypercalcemia],[84],[102],[21],[34],0,0
3,METHODS: We report two cases of pseudoporphyri...,"[naproxen, oxaprozin]","[pseudoporphyria, pseudoporphyria]","[58, 71]","[66, 80]",[32],[47],0,0
4,"Naproxen, the most common offender, has been a...",[Naproxen],[erythropoietic protoporphyria],[0],[8],[134],[163],0,0
...,...,...,...,...,...,...,...,...,...
4266,Reversible nonthrombocytopenic palpable purpur...,[metoclopramide],[nonthrombocytopenic palpable purpura],[64],[78],[11],[47],0,0
4267,Lithium treatment was terminated in 1975 becau...,"[Lithium, lithium]","[diabetes insipidus-like syndrome, diabetes in...","[0, 52]","[7, 59]",[80],[112],0,0
4268,Eosinophilia caused by clozapine was observed ...,[clozapine],[Eosinophilia],[23],[32],[0],[12],0,0
4269,Eosinophilia has been encountered from 0.2 to ...,[clozapine],[Eosinophilia],[55],[64],[0],[12],0,0


In [ ]:
import fnmatch


In [ ]:
data_dict = df.to_dict('index')
data_dict[0]


{'text': 'Intravenous azithromycin induced ototoxicity.',
 'drug': ['azithromycin'],
 'effect': ['ototoxicity'],
 'drug_indices_start': [12],
 'drug_indices_end': [24],
 'effect_indices_start': [33],
 'effect_indices_end': [44],
 'len_effect': 0,
 'len_drug': 0}

In [ ]:
for key,value in data_dict.items():
  
  data_dict[key]['text'] = data_dict[key]['text'].replace('-', ' ')
  for index,drug in enumerate(data_dict[key]['drug']):
    if '-' in drug:
      data_dict[key]['drug'][index] = drug.replace('-', ' ')

  for index,effect in enumerate(data_dict[key]['effect']):
    if '-' in effect:
      data_dict[key]['effect'][index] = effect.replace('-', ' ')

  if "We conjectured that the side effects of insulin" in data_dict[key]['text']:
    print(data_dict[key])

  item_copy = data_dict[key]
  for i,drug in enumerate(item_copy['drug_indices_start']):
        for j,event in enumerate(item_copy['effect_indices_start']):
            xs = set()
            try:
              x = range(item_copy['drug_indices_start'][i],item_copy['drug_indices_end'][i])
              y = range(item_copy['effect_indices_start'][j],item_copy['effect_indices_end'][j])
            except:
              print((item_copy))
            xs = set(x)
            z = xs.intersection(y)

            if z:
              print(z)
              del data_dict[key]['effect_indices_start'][j]
              del data_dict[key]['effect_indices_end'][j]

{'text': 'We conjectured that the side effects of insulin, such as anti natriuresis and increased vascular permeability, might be pronounced in the presence of the hepatic dysfunction that accompanies insulin insensitivity, hyperinsulinaemia and hypoalbuminaemia.', 'drug': ['insulin'], 'effect': ['anti natriuresis'], 'drug_indices_start': [40, 191, 219], 'drug_indices_end': [47, 198, 226], 'effect_indices_start': [57], 'effect_indices_end': [73], 'len_effect': 0, 'len_drug': 0}
{'text': 'CONCLUSIONS: Low dosages of quinacrine used for malaria prophylaxis can be associated with a delayed, severe maculopathy indistinguishable from chloroquine maculopathy in certain patients.', 'drug': ['chloroquine', 'quinacrine'], 'effect': ['maculopathy', 'severe maculopathy'], 'drug_indices_start': [28, 144], 'drug_indices_end': [38, 155], 'effect_indices_start': [102, 156], 'effect_indices_end': [120], 'len_effect': -1, 'len_drug': 0}
{'text': 'CONCLUSIONS: Low dosages of quinacrine used for malaria 

In [ ]:
df = pd.DataFrame(list(data_dict.values()))

In [ ]:
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example

docs=[]

for value,content in data_dict.items():
  try:
    doc = nlp(content['text'])
    gold_dict = {"entities": 
                [(content['drug_indices_start'][i], content['drug_indices_end'][i], "DRUG") for i in range(len(content['drug_indices_start']))] +
                [(content['effect_indices_start'][j], content['effect_indices_end'][j], "EFFECT") for j in range(len(content['effect_indices_start']))] }
    example = Example.from_dict(doc, gold_dict)
  except Exception as e:
    print(e,content['text'])
    pass
  docs.append(doc)


list index out of range OBJECTIVE: The purpose of this study was to determine whether desensitization to NPH insulin, as well as standard insulin desensitization, could control allergic symptoms in a patient allergic to both NPH and regular insulin.


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "With the negative viral serologies, the clinical p..." with entities "[(139, 151, 'DRUG'), (96, 123, 'EFFECT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "We report 2 patients who developed polyarteritis n..." with entities "[(66, 97, 'DRUG'), (35, 55, 'EFFECT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/trai

list index out of range CONCLUSIONS: Low dosages of quinacrine used for malaria prophylaxis can be associated with a delayed, severe maculopathy indistinguishable from chloroquine maculopathy in certain patients.
list index out of range The clinical picture was identical to that of chloroquine and hydroxychloroquine maculopathy.


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "We report five cases of carboplatin (CBDCA) hypers..." with entities "[(24, 35, 'DRUG'), (37, 42, 'DRUG'), (83, 93, 'DRU...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


[E103] Trying to set conflicting doc.ents: '(123, 148, 'DRUG')' and '(133, 151, 'DRUG')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead. We review the literature on previously reported cases of cutaneous necrosis after injection of standard interferon alfa or pegylated interferon alfa 2b and discuss the different pathophysiologic mechanisms that might be involved.


...

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "Toxic epidermal necrolysis resulted after 19 days ..." with entities "[(68, 84, 'DRUG'), (89, 103, 'DRUG'), (0, 26, 'EFF...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "Neurotoxicity of intrathecal administration of amp..." with entities "[(47, 61, 'DRUG'), (0, 13, 'EFFECT')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages

list index out of range CASE: We report a case of a woman with severe human insulin induced lipoatrophy who has been treated exclusively with recombinant DNA human insulin since the onset of IDDM.


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "Acute nitrite toxicity results from industrial exp..." with entities "[(6, 13, 'DRUG'), (102, 109, 'DRUG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


list index out of range The toxicity and plasma concentrations of ifosfamide, chloroacetaldehyde, and 4 hydroxyifosfamide were then determined over 24 h after a single 1.6 g/m2 dose of ifosfamide.
list index out of range In case 1, a total daily dose of 25 mg sertraline, with nondetectable sertraline and desmethylsertraline blood levels, resulted in a doubling of the lamotrigine blood level with symptoms of toxicity.


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "Although major hazards of treatment of hypophospha..." with entities "[(43, 52, 'DRUG'), (74, 83, 'DRUG'), (88, 98, 'DRU...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "High dose phosphate treatment leads to hypokalemia..." with entities "[(10, 19, 'DRUG'), (58, 67, 'DRUG'), (39, 50, 'EFF...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3

In [ ]:
TRAIN_DATA = []
for value,content in data_dict.items():
  try:
    text = content['text']
    TRAIN_DATA.append({"text":text,"entities": 
                [(content['drug_indices_start'][i], content['drug_indices_end'][i], "DRUG") for i in range(len(content['drug_indices_start']))] +
                [(content['effect_indices_start'][j], content['effect_indices_end'][j], "EFFECT") for j in range(len(content['effect_indices_start']))]})
  except Exception as e:
    print(e,content['text'])
    pass

In [ ]:
TRAIN_DATA[456]

{'text': 'We describe a patient who experienced a lichenoid eruption after the initiation of salsalate for relief of arthritic pain.',
 'entities': [(83, 92, 'DRUG'), (40, 58, 'EFFECT')]}

In [ ]:
nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin()

In [ ]:
from tqdm import tqdm
from spacy.util import filter_spans

for training_example  in tqdm(TRAIN_DATA[3186:]): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("test.spacy")

 69%|██████▊   | 727/1061 [00:00<00:00, 3682.25it/s]

Skipping entity
Skipping entity


100%|██████████| 1061/1061 [00:00<00:00, 3583.91it/s]


Skipping entity
Skipping entity
Skipping entity


In [ ]:
!python -m spacy train config_spacy.cfg --output ./ --paths.train train.spacy --paths.dev test.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-10-21 15:21:37,280] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-10-21 15:21:37,289] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-10-21 15:21:37,293] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-10-21 15:21:37,294] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are

In [ ]:
import spacy
nlp = spacy.load("model-last")

In [ ]:
text = ['''Background: Prophylactic use of amoxicillin and amoxicillin/clavulanic acid, although controversial, is common in routine clinical practice in third molar surgery.

Material and methods: Our objective was to assess the efficacy of prophylactic amoxicillin with or without clavulanic acid in reducing the incidence of dry socket and/or infection after third molar extraction. We conducted a systematic review and meta-analysis consulting electronic databases and references in retrieved articles. We included double-blind placebo-controlled randomized clinical trials published up to June 2015 investigating the efficacy of amoxicillin with or without clavulanic acid on the incidence of the aforementioned conditions after third molar extraction. Relative risks (RRs) were estimated with a generic inverse-variance approach and a random effect model using Stata/IC 13 and Review Manager Version 5.2. Stratified analysis was performed by antibiotic type.

Results: We included 10 papers in the qualitative review and in the quantitative synthesis (1997 extractions: 1072 in experimental groups and 925 in controls, with 27 and 74 events of dry socket and/or infection, respectively). The overall RR was 0.350 (p<0.001; 95% CI 0.214 to 0.574). We found no evidence of heterogeneity (I2=0%, p=0.470). The number needed to treat was 18 (95% CI 13 to 29). Five studies reported adverse reactions (RR=1.188, 95% CI 0.658 to 2.146, p =0.567). The RRs were 0.563 for amoxicillin (95% CI 0.295 to 1.08, p=0.082) and 0.215 for amoxicillin/clavulanic acid (95% CI 0.117 to 0.395, p<0.001).

Conclusions: Prophylactic use of amoxicillin does not significantly reduce the risk of infection and/or dry socket after third molar extraction. With amoxicillin/clavulanic acid, the risk decreases significantly. Nevertheless, considering the number needed to treat, low prevalence of infection, potential adverse reactions to antibiotics and lack of serious complications in placebo groups, the routine prescription of amoxicillin with or without clavulanic acid is not justified.''']

In [ ]:
for doc in nlp.pipe(text):
    print([(ent.text, ent.label_) for ent in doc.ents])

[('amoxicillin', 'DRUG'), ('amoxicillin', 'DRUG'), ('clavulanic acid', 'DRUG'), ('amoxicillin', 'DRUG'), ('clavulanic acid', 'DRUG'), ('infection', 'EFFECT'), ('amoxicillin', 'DRUG'), ('clavulanic acid', 'DRUG'), ('dry socket', 'EFFECT'), ('infection', 'EFFECT'), ('amoxicillin', 'DRUG'), ('amoxicillin', 'DRUG'), ('clavulanic acid', 'DRUG'), ('amoxicillin', 'DRUG'), ('dry socket', 'EFFECT'), ('amoxicillin', 'DRUG'), ('clavulanic acid', 'DRUG'), ('amoxicillin', 'DRUG'), ('clavulanic acid', 'DRUG')]


In [ ]:
for doc in nlp.pipe(text, disable=["tagger"]):
   print(f"spans: {[(e.start, e.text, e.label_) for e in doc.ents]}")

spans: [(5, 'amoxicillin', 'DRUG'), (7, 'amoxicillin', 'DRUG'), (9, 'clavulanic acid', 'DRUG'), (40, 'amoxicillin', 'DRUG'), (44, 'clavulanic acid', 'DRUG'), (54, 'infection', 'EFFECT'), (98, 'amoxicillin', 'DRUG'), (102, 'clavulanic acid', 'DRUG'), (189, 'dry socket', 'EFFECT'), (192, 'infection', 'EFFECT'), (267, 'amoxicillin', 'DRUG'), (281, 'amoxicillin', 'DRUG'), (283, 'clavulanic acid', 'DRUG'), (302, 'amoxicillin', 'DRUG'), (312, 'dry socket', 'EFFECT'), (320, 'amoxicillin', 'DRUG'), (322, 'clavulanic acid', 'DRUG'), (362, 'amoxicillin', 'DRUG'), (366, 'clavulanic acid', 'DRUG')]


In [ ]:
df.to_json("dataset.jsonl", orient="records", lines=True)

In [ ]:
cons_dataset = load_dataset("json", data_files="dataset.jsonl")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
cons_dataset
from datasets import DatasetDict

In [ ]:
cons_dataset = cons_dataset["train"].train_test_split(test_size=0.2)



In [ ]:
cons_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'effect', 'drug_indices_start', 'drug_indices_end', 'effect_indices_start', 'effect_indices_end', 'len_effect', 'len_drug'],
        num_rows: 3397
    })
    test: Dataset({
        features: ['text', 'drug', 'effect', 'drug_indices_start', 'drug_indices_end', 'effect_indices_start', 'effect_indices_end', 'len_effect', 'len_drug'],
        num_rows: 850
    })
})

In [ ]:
from datasets import Dataset, ClassLabel, Sequence, load_dataset, load_metric

label_list = ['O', 'B-DRUG', 'I-DRUG', 'B-EFFECT', 'I-EFFECT']

custom_seq = Sequence(feature=ClassLabel(num_classes=5, 
                                         names=label_list,
                                         names_file=None, id=None), length=-1, id=None)

cons_dataset["train"].features["ner_tags"] = custom_seq
cons_dataset["test"].features["ner_tags"] = custom_seq

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b1bfb0bcaf9ab193ff3ad01e87732fc1/config.json
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--allenai--scibert_scivocab_uncased/snapshots/24f92d32b

In [ ]:
def generate_row_labels(row, verbose=False):
    """ Given a row from the consolidated `Ade_corpus_v2_drug_ade_relation` dataset, 
    generates BIO tags for drug and effect entities. 
    
    """

    text = row["text"]

    labels = []
    label = "O"
    prefix = ""
    
    # while iterating through tokens, increment to traverse all drug and effect spans
    drug_index = 0
    effect_index = 0
    
    tokens = tokenizer(text, return_offsets_mapping=True, padding=True, truncation=True)
    print(tokenizer.convert_ids_to_tokens(tokens["input_ids"]))

    for n in range(len(tokens["input_ids"])):
        offset_start, offset_end = tokens["offset_mapping"][n]

        # should only happen for [CLS] and [SEP]
        if offset_end - offset_start == 0:
            labels.append(-100)
            continue
        
        if drug_index < len(row["drug_indices_start"]) and offset_start == row["drug_indices_start"][drug_index]:
            label = "DRUG"
            prefix = "B-"

        elif effect_index < len(row["effect_indices_start"]) and offset_start == row["effect_indices_start"][effect_index]:
            label = "EFFECT"
            prefix = "B-"
        
        labels.append(label_list.index(f"{prefix}{label}"))
            
        if drug_index < len(row["drug_indices_end"]) and offset_end == row["drug_indices_end"][drug_index]:
            label = "O"
            prefix = ""
            drug_index += 1
            
        elif effect_index < len(row["effect_indices_end"]) and offset_end == row["effect_indices_end"][effect_index]:
            label = "O"
            prefix = ""
            effect_index += 1

        # need to transition "inside" if we just entered an entity
        if prefix == "B-":
            prefix = "I-"
    
    if verbose:
        print(f"{row}\n")
        orig = tokenizer.convert_ids_to_tokens(tokens["input_ids"])
        for n in range(len(labels)):
            print(orig[n], labels[n])
    tokens["labels"] = labels
    
    return tokens

In [ ]:
cons_dataset["train"][1]

{'text': 'A case of a 53-year-old man who developed acute pneumonitis after bleomycin and moderate oxygen administration is presented.',
 'drug': ['bleomycin', 'oxygen'],
 'effect': ['acute pneumonitis', 'acute pneumonitis'],
 'drug_indices_start': [66, 89],
 'drug_indices_end': [75, 95],
 'effect_indices_start': [42],
 'effect_indices_end': [59],
 'len_effect': 0,
 'len_drug': 0}

In [ ]:
generate_row_labels(cons_dataset["train"][50], verbose=True)

['[CLS]', 'one', 'of', 'these', 'was', 'clo', '##fa', '##zi', '##mine', ',', 'an', 'an', '##ilin', '##e', 'apo', '##sa', '##fr', '##anine', 'derivative', 'known', 'to', 'produce', 'a', 'cer', '##oid', '-', 'like', 'pigment', 'in', 'the', 'tissues', 'of', 'patients', 'treated', 'with', 'this', 'drug', 'or', 'lep', '##roma', '##to', '##us', 'lep', '##ros', '##y', '.', '[SEP]']
{'text': 'One of these was clofazimine, an aniline aposafranine derivative known to produce a ceroid-like pigment in the tissues of patients treated with this drug or lepromatous leprosy.', 'drug': ['clofazimine'], 'effect': ['ceroid-like pigment'], 'drug_indices_start': [17], 'drug_indices_end': [28], 'effect_indices_start': [84], 'effect_indices_end': [103], 'len_effect': 0, 'len_drug': 0}

[CLS] -100
one 0
of 0
these 0
was 0
clo 1
##fa 2
##zi 2
##mine 2
, 0
an 0
an 0
##ilin 0
##e 0
apo 0
##sa 0
##fr 0
##anine 0
derivative 0
known 0
to 0
produce 0
a 0
cer 3
##oid 4
- 4
like 4
pigment 4
in 0
the 0
tissues 0
of 0
p

{'input_ids': [102, 482, 131, 407, 241, 25091, 4295, 5889, 22229, 422, 130, 130, 16863, 30107, 20497, 4172, 3153, 9451, 7192, 1721, 147, 3299, 106, 1866, 1153, 579, 1967, 13139, 121, 111, 3821, 131, 568, 2338, 190, 238, 1698, 234, 29005, 27366, 861, 153, 29005, 641, 30126, 205, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 3), (4, 6), (7, 12), (13, 16), (17, 20), (20, 22), (22, 24), (24, 28), (28, 29), (30, 32), (33, 35), (35, 39), (39, 40), (41, 44), (44, 46), (46, 48), (48, 53), (54, 64), (65, 70), (71, 73), (74, 81), (82, 83), (84, 87), (87, 90), (90, 91), (91, 95), (96, 103), (104, 106), (107, 110), (111, 118), (119, 121), (122, 130), (131, 138), (139, 143), (144, 148), (149

In [ ]:
labeled_dataset = cons_dataset.map(generate_row_labels)

  0%|          | 0/3397 [00:00<?, ?ex/s]

  0%|          | 0/850 [00:00<?, ?ex/s]

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "allenai/scibert_scivocab_uncased"
batch_size = 8

In [ ]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.05,
    logging_steps=1
)

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.7 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=fb56f4f6c1e040f4c4c47bff403171c504b91757742bdf91a9df4e42406a77b1
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
metric = load_metric("seqeval")

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=labeled_dataset["train"],
    eval_dataset=labeled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 

)

In [ ]:

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: len_drug, drug, offset_mapping, effect_indices_start, effect, len_effect, drug_indices_end, drug_indices_start, effect_indices_end, text. If len_drug, drug, offset_mapping, effect_indices_start, effect, len_effect, drug_indices_end, drug_indices_start, effect_indices_end, text are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3397
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) =

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.089400,0.124416,0.825445,0.894588,0.858627,0.957613
2,0.046900,0.111643,0.851291,0.899765,0.874857,0.963219
3,0.014400,0.123332,0.861731,0.909176,0.884818,0.963774
4,0.015400,0.127067,0.863860,0.907765,0.885268,0.965720
5,0.007700,0.136413,0.858923,0.908235,0.882891,0.965627


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: len_drug, drug, offset_mapping, effect_indices_start, effect, len_effect, drug_indices_end, drug_indices_start, effect_indices_end, text. If len_drug, drug, offset_mapping, effect_indices_start, effect, len_effect, drug_indices_end, drug_indices_start, effect_indices_end, text are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 850
  Batch size = 8
Saving model checkpoint to scibert_scivocab_uncased-finetuned-ner/checkpoint-500
Configuration saved in scibert_scivocab_uncased-finetuned-ner/checkpoint-500/config.json
Model weights saved in scibert_scivocab_uncased-finetuned-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in scibert_scivocab_uncased-finetuned-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in scibert_scivoc

TrainOutput(global_step=2125, training_loss=0.09772575855397564, metrics={'train_runtime': 295.1191, 'train_samples_per_second': 57.553, 'train_steps_per_second': 7.2, 'total_flos': 413011649910990.0, 'train_loss': 0.09772575855397564, 'epoch': 5.0})

In [ ]:
from transformers import pipeline
effect_ner_model = pipeline(task="ner", model=model, tokenizer=tokenizer,aggregation_strategy='max', device=0)


In [ ]:
effect_ner_model(labeled_dataset["test"][4]["text"])

[{'entity_group': 'LABEL_0',
  'score': 0.9996493,
  'word': 'we suggest discontinuing',
  'start': 0,
  'end': 24},
 {'entity_group': 'LABEL_1',
  'score': 0.99880373,
  'word': 'gemcitabine',
  'start': 25,
  'end': 36},
 {'entity_group': 'LABEL_0',
  'score': 0.9994802,
  'word': 'if',
  'start': 37,
  'end': 39},
 {'entity_group': 'LABEL_3',
  'score': 0.99278295,
  'word': 'radiation',
  'start': 40,
  'end': 49},
 {'entity_group': 'LABEL_4',
  'score': 0.99557805,
  'word': 'recall',
  'start': 50,
  'end': 56},
 {'entity_group': 'LABEL_0',
  'score': 0.99916106,
  'word': 'is observed.',
  'start': 57,
  'end': 69}]

In [ ]:
from spacy import displacy
def visualize_entities(sentence):
    tokens = effect_ner_model(sentence)
    entities = []
    
    for token in tokens:
        label = int(token["entity_group"][-1])
        if label != 0:
            token["label"] = label_list[label]
            entities.append(token)
    
    params = [{"text": sentence,
               "ents": entities,
               "title": None}]


    
    html = displacy.render(params, style="ent", jupyter=True, manual=True, options={
        "colors": {
                   "B-DRUG": "#f08080",
                   "I-DRUG": "#f08080",
                   "B-EFFECT": "#9bddff",
                   "I-EFFECT": "#9bddff",
               },
    })
    

In [ ]:
effect_ner_model("Acetaminophen/paracetamol is the most widely used drug of the world. At the same time, it is probably one of the most dangerous compounds in medical use, causing hundreds of deaths in all industrialized countries due to acute liver failure (ALF). Publications of the last 130 years found in the usual databases were analyzed. Personal contacts existed to renowned researchers having contributed to the medical use of paracetamol and its precursors as H.U. Zollinger, S. Moeschlin, U. Dubach, J. Axelrod and others. Further information is found in earlier reviews by Eichengrün, Rodnan and Benedek, Sneader, Brune; comp. references. The history of the discovery of paracetamol starts with an error (active against worms), continues with a false assumption (paracetamol is safer than phenacetin), describes the first side-effect 'epidemy' (phenacetin nephropathy, drug-induced interstitial nephritis) and ends with the discovery of second-generation problems due to the unavoidable production of a highly toxic metabolite of paracetamol N-acetyl-p-benzoquinone imine (NAPQI) that may cause not only ALF and kidney damage but also impaired development of the fetus and the newborn child. It appears timely to reassess the risk/benefit ratio of this compound.")

[{'entity_group': 'LABEL_2',
  'score': 0.9983443,
  'word': 'acetaminophen',
  'start': 0,
  'end': 13},
 {'entity_group': 'LABEL_0',
  'score': 0.9992754,
  'word': '/',
  'start': 13,
  'end': 14},
 {'entity_group': 'LABEL_2',
  'score': 0.9991585,
  'word': 'paracetamol',
  'start': 14,
  'end': 25},
 {'entity_group': 'LABEL_0',
  'score': 0.99793786,
  'word': 'is the most widely used drug of the world. at the same time, it is probably one of the most dangerous compounds in medical use, causing hundreds of deaths in all industrialized countries due to',
  'start': 26,
  'end': 219},
 {'entity_group': 'LABEL_3',
  'score': 0.9463674,
  'word': 'acute',
  'start': 220,
  'end': 225},
 {'entity_group': 'LABEL_4',
  'score': 0.9220505,
  'word': 'liver failure',
  'start': 226,
  'end': 239},
 {'entity_group': 'LABEL_0',
  'score': 0.99964476,
  'word': '(',
  'start': 240,
  'end': 241},
 {'entity_group': 'LABEL_3',
  'score': 0.5279048,
  'word': 'alf',
  'start': 241,
  'end': 244}

In [ ]:


visualize_entities("""I got a severe headache while I was under paracetamol treatment. But my stomach was fine.""")


In [ ]:
from spacy.tokens import Doc
from spacy.training import Example

list_examples=[]

for i,text in enumerate(cons_dataset['train']):
  item = text
  try:
    doc = nlp(item['text'])
  except Exception as e:
    print(e)
    print(doc,i)
    print()


  try:
    gold_dict = {"entities": 
      [(item['drug_indices_start'][i],item['drug_indices_end'][i], "DRUG") for i in range(len(item['drug_indices_start']))] +
      [(item['effect_indices_start'][i],item['effect_indices_end'][i], "EFFECT") for i in range(len(item['effect_indices_start']))]}
  except Exception as e:
    print(e)
    print(doc,i)
    print()

  try:
    example = Example.from_dict(doc, gold_dict)
  except Exception as e:
    print(e)
    print(doc,i)
    print()

  list_examples.append(example)


[E103] Trying to set conflicting doc.ents: '(98, 131, 'EFFECT')' and '(122, 132, 'EFFECT')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.
Eleven patients developed infection requiring hospitalization while taking leflunomide including: lower respiratory tract infections (3), cellulitis (2), disseminated herpes zoster (2), probable TB liver (1), abdominal sepsis (1), mycotic aneurysm (1) and gastroenteritis (1). 930

[E103] Trying to set conflicting doc.ents: '(123, 148, 'DRUG')' and '(133, 151, 'DRUG')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap. To work with overlapping entities, consider using doc.spans instead.
We review the literature on previously reported cases of cutaneous necrosis after injection of standard interferon alfa or pegylated interferon alfa-2b and discuss the different pathophysiologic mechanisms

In [ ]:
example 

{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DRUG', 'L-DRUG', 'O', 'O', 'O'], 'links': {}}, 'token_annotation': {'ORTH': ['This', 'paper', 'reports', 'a', 'case', 'of', 'fatal', 'perhexiline', 'maleate', 'liver', 'injury', '.'], 'SPACY': [True, True, True, True, True, True, True, True, True, True, False, False], 'TAG': ['', '', '', '', '', '', '', '', '', '', '', ''], 'LEMMA': ['', '', '', '', '', '', '', '', '', '', '', ''], 'POS': ['', '', '', '', '', '', '', '', '', '', '', ''], 'MORPH': ['', '', '', '', '', '', '', '', '', '', '', ''], 'HEAD': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'DEP': ['', '', '', '', '', '', '', '', '', '', '', ''], 'SENT_START': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}}

In [ ]:
from spacy.tokens import DocBin

db = DocBin(docs=[ex.reference for ex in [example]])
db.to_disk("/train.spacy")

In [ ]:
from spacy.tokens import Doc
from spacy.training import Example

list_examples=[]

for i,text in enumerate(cons_dataset['test']):
  item = text
  try:
    doc = nlp(item['text'])
  except Exception as e:
    print(e)
    print(doc,i)
    print()


  try:
    gold_dict = {"entities": 
      [(item['drug_indices_start'][i],item['drug_indices_end'][i], "DRUG") for i in range(len(item['drug_indices_start']))] +
      [(item['effect_indices_start'][i],item['effect_indices_end'][i], "EFFECT") for i in range(len(item['effect_indices_start']))]}
  except Exception as e:
    print(e)
    print(doc,i)
    print()

  try:
    example = Example.from_dict(doc, gold_dict)
  except Exception as e:
    print(e)
    print(doc,i)
    print()

  list_examples.append(example)


In [ ]:
db = DocBin(docs=[ex.reference for ex in [example]])
db.to_disk("/test.spacy")

In [ ]:
from spacy.tokens import Doc
from spacy.training import Example

list_examples=[]

for i,text in enumerate(cons_dataset['dev']):
  item = text
  try:
    doc = nlp(item['text'])
  except Exception as e:
    print(e)
    print(doc,i)
    print()


  try:
    gold_dict = {"entities": 
      [(item['drug_indices_start'][i],item['drug_indices_end'][i], "DRUG") for i in range(len(item['drug_indices_start']))] +
      [(item['effect_indices_start'][i],item['effect_indices_end'][i], "EFFECT") for i in range(len(item['effect_indices_start']))]}
  except Exception as e:
    print(e)
    print(doc,i)
    print()

  try:
    example = Example.from_dict(doc, gold_dict)
  except Exception as e:
    print(e)
    print(doc,i)
    print()

  list_examples.append(example)

In [ ]:
db = DocBin(docs=[ex.reference for ex in [example]])
db.to_disk("/dev.spacy")

In [ ]:
!python -m spacy project clone tutorials/rel_component

✔ Cloned 'tutorials/rel_component' from 'explosion/projects' (branch
'v3')
/content/rel_component
✔ Your project is now ready!
To fetch the assets, run:
python -m spacy project assets /content/rel_component


In [ ]:
!cd rel_component

In [ ]:
! ls

assets	 data		rel_component  scripts
configs  dataset.jsonl	sample_data    training
